In [ ]:
from tiff_utils import *

from collections import OrderedDict
from shapely.ops import transform
import pandas as pd
import numpy as np
import shapely.wkt
import geopandas

In [ ]:
hail_data = pd.read_csv('../large_hail_events.csv').drop(columns='Unnamed: 0').set_index('id')

In [ ]:
hail_data['cm'] = hail_data.max_hail_diameter.str.split().apply(lambda x: int(x[0]) if isinstance(x, list) else x).fillna(0)

In [ ]:
my_points = [
    (45.10, 10.98),
    (45.10, 11.14),
    (44.99, 11.14),
    (44.99, 10.98)
]

In [ ]:
hail_data[(hail_data.latitude <= my_points[0][0]) &
          (hail_data.latitude >= my_points[2][0]) &
          (hail_data.longitude >= my_points[0][1]) &
          (hail_data.longitude <= my_points[2][1])].sort_values('cm', ascending=False).head(10)

In [ ]:
# Copernicus hub likes polygons in lng/lat format
mygeojson = geopandas.GeoSeries([get_test_polygon(my_points, swap_coordinates=True)]).__geo_interface__

In [ ]:
# connect to the API
from sentinelsat import SentinelAPI, geojson_to_wkt
from datetime import date

api = SentinelAPI('pipporusso', 'pipporusso89', 'https://scihub.copernicus.eu/dhus')

In [ ]:
# download single scene by known product id
# api.download(<product_id>)

# search by polygon, time, and SciHub query keywords
footprint = geojson_to_wkt(mygeojson)
products = api.query(footprint,
                     # date=(date(2022, 5, 1), date(2022, 8, 20)),
                     date=(date(2021, 5, 1), date(2021, 8, 20)),
                     platformname='Sentinel-2',
                     processinglevel='Level-2A',
                     cloudcoverpercentage=(0, 30))

In [ ]:
my_poly = get_test_polygon(my_points)

In [ ]:
filtered = OrderedDict()
for pr in products.items():
    footprint = pr[1]['footprint']
    P = shapely.wkt.loads(footprint)       # convert string to Polygon
    P = transform(lambda x, y: (y, x), P)  # swap coordinates
    if my_poly.difference(P).is_empty:
        filtered[pr[0]] = pr[1]
        try:
            # if document is not online raise exception (but starts the upload request)
            api.download(pr[0], directory_path='/Users/beppe/Projects/AgriTech/Sentinel-2/2021')
        except:
            print(pr[0])
            continue